<a href="https://colab.research.google.com/github/lemmingy/My_pipelines/blob/main/liner_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install category_encoders

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/Kaggle/Library/Input_data/liner_model

/content/drive/My Drive/Colab Notebooks/Kaggle/Library/Input_data/liner_model


In [3]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("train.csv")
df_train.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Respondent,ConvertedSalary,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,...,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,3,70841.0,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,...,4.0,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
1,7,21426.0,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,...,6.0,1.0,7.0,5.0,Algorithms making important decisions,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, but only within the company",Upper management at the company/organization,Yes,10 (Very Likely),Daily or almost daily,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Before 5:00 AM,Over 12 hours,1 - 2 hours,Never,NaN,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
2,8,41671.0,Yes,No,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",

In [4]:
# Testのデータの傾向をよくみる。Trainingデータと比べて違う所はないか？見たことのない国を当てるグループパーティションや、時系列スプリットなど。
df_test = pd.read_csv("test.csv")
df_test.head()

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,...,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,81,No,Yes,Germany,No,Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",100 to 499 employees,Back-end developer;Desktop or enterprise appli...,9-11 years,9-11 years,Moderately satisfied,Extremely satisfied,Working as a product manager or project manager,"I’m not actively looking, but I am open to new...",Less than a year ago,9.0,5.0,10.0,2.0,3.0,4.0,6.0,1.0,8.0,7.0,1.0,10.0,11.0,6.0,8.0,9.0,2.0,5.0,4.0,3.0,7.0,5.0,1.0,...,3.0,2.0,6.0,5.0,NaN,NaN,Nobody,"I don't care about it, or I haven't thought ab...",Depends on what it is,"Yes, but only within the company",Upper management at the company/organization,Unsure / I don't know,4,A few times per week,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,A little bit interested,Somewhat interested,Not at all interested,Not at all interested,Not at all interested,Between 6:01 - 7:00 AM,5 - 8 hours,1 - 2 hours,Never,NaN,I don't typically exercise,Male,Straight or heterosexual,"Secondary school (e.g. American high school, G...",White or of European descent,35 - 44 years old,Yes,NaN,The survey was too long,Neither easy nor difficult
1,85,Yes,Yes,France,"Yes, part-time",Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",10 to 19 employees,Back-end developer;Front-end developer;Student,9-11 years,0-2 years,Neither satisfied nor dissatisfied,Moderately dissatisfied,Working as a founder or co-founder of my own c...,"I’m not actively looking, but I am open to new...",Less than a year ago,1.0,10.0,4.0,5.0,2.0,6.0,3.0,7.0,8.0,9.0,1.0,3.0,9.0,11.0,10.0,8.0,4.0,7.0,6.0,2.0,5.0,5.0,1.0,...,5.0,4.0,7.0,2.0,"Evolving definitions of ""fairness"" in algorith...",Artificial intelligence surpassing human intel...,Nobody,I'm excited about the possibilities more than ...,No,Depends on what it is,NaN,Yes,10 (Very Likely),Multiple times per day,No,NaN,Yes,NaN,5,I'm not sure,Extremely interested,Extremely interested,A little bit interested,Very interested,Very interested,Between 5:00 - 6:00 AM,Over 12 hours,1 - 2 hours,Daily or almost every day,NaN,I don't typically exercise,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,18 - 24 years old,No,NaN,The survey was too long,Very easy
2,92,Yes,Yes,Germany,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",500 to 999 employees,Full-stack developer,6-8 years,0-2 years,Slightly dissatisfied,Slightly satisfied,Doing the same work,I am actively looking for a job,Between 1 and 2 years ago,6.0,2.0,8.0,1.0,3.0,5.0,10.0,4.0,9.0,7.0,1.0,9.0,2.0,7.0,10.0,3.0,11.0,4.0,8.0,5.

In [5]:
# 時間特徴量の処理（あれば）
def time_encode(df, col):
    # この方法だと場合によって最大値が変化するデータでは正確な値は出ない
    # 例：月の日数が30日や31日の場合がある
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    return df


In [6]:
# 数値カラム、カテゴリカラム、テキストコラムのリストを取得
def get_columns(df):
  lis_cols_num = [ col for col in df.columns if df[col].dtype == "float64" or df[col].dtype == "int64" ]
  lis_cols_cat = [ col for col in df.columns if df[col].dtype == "object" ]
  return lis_cols_num, lis_cols_cat

lis_cols_num, lis_cols_cat  =  get_columns(df_train)

In [7]:
# 数値カラムの外れ値の影響を減らすためのクリッピング
def clip(df, lis_cols_num):
  for col in lis_cols_num:
    upperbound, lowerbound= np.percentile(df[col],[1,99])
    df[col]=np.clip(df[col],upperbound,lowerbound)
    return df

df_train = clip(df_train, lis_cols_num)
df_train.head()

,Respondent,ConvertedSalary,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,...,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1073.36,70841.0,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,...,4.0,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
1,1073.36,21426.0,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,...,6.0,1.0,7.0,5.0,Algorithms making important decisions,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, but only within the company",Upper management at the company/organization,Yes,10 (Very Likely),Daily or almost daily,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Before 5:00 AM,Over 12 hours,1 - 2 hours,Never,NaN,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
2,1073.36,41671.0,Yes,No,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engine

In [18]:
#カテゴリ列について、trainingにしか現れないカテゴリ、もしくはtestにしか現れないカテゴリを可視化
def check_cat(df_train, df_test, col):
  train_s = set(df_train[col].unique())
  test_s = set(df_test[col].unique())
  only_training = train_s - test_s
  only_test = test_s - train_s
  print("trainingのみ:"+str(only_training))
  print("testのみ:"+str(only_test))
  return only_training, only_test


only_training, only_test = check_cat(df_train, df_test, "Country")

trainingのみ:{'United Republic of Tanzania', 'Lebanon', 'Eritrea', 'Honduras', 'Slovakia', 'Rwanda', 'Namibia', 'Canada', 'Uzbekistan', 'Mozambique', 'Turkey', 'Dominica', 'Ukraine', 'India', 'Guatemala', 'Jamaica', 'Libyan Arab Jamahiriya', 'China', 'Kenya', 'Turkmenistan', 'United Kingdom', 'United Arab Emirates', 'Poland', 'Russian Federation', 'Kuwait', 'Afghanistan', 'Latvia', 'Hong Kong (S.A.R.)', 'Spain', 'Maldives', 'Botswana', 'Colombia', 'Albania', 'Cyprus', 'Slovenia', 'Nicaragua', 'Ecuador', 'Myanmar', 'Viet Nam', 'Pakistan', 'Czech Republic', 'Syrian Arab Republic', 'Republic of Moldova', 'Gambia', 'Saudi Arabia', 'Philippines', 'Luxembourg', 'Taiwan', 'Iraq', 'El Salvador', 'Belgium', 'Sweden', 'Democratic Republic of the Congo', 'Somalia', 'Switzerland', 'Romania', 'Lesotho', 'Argentina', 'Qatar', 'Benin', 'Thailand', 'Singapore', 'Republic of Korea', 'Other Country (Not Listed Above)', 'Bahamas', 'Serbia', 'Malawi', 'New Zealand', 'Iceland', 'Mexico', 'Senegal', 'South Ko

In [9]:
#以下のセルの準備。本来数字のカテゴリ特徴量を数字に変換する準備 #testと見比べる。testに無いものはOthersに落とす。
df_train["CompanySize"].unique()

array(['10,000 or more employees', '10 to 19 employees',
       '100 to 499 employees', '20 to 99 employees',
       '1,000 to 4,999 employees', '500 to 999 employees',
       '5,000 to 9,999 employees', 'Fewer than 10 employees', nan],
      dtype=object)

In [10]:
#本来数字であるカテゴリ特徴量を数字に変換する。Testに出てこないカテゴリをOthersに落とす。
def order_num(df, col, lis):
  now_lis = df[col].unique()
  dic = {}
  for a, b in zip(now_lis, lis):
    dic[a] = b
  df[col].replace(dic, inplace = True)
  return df

order_lis=[10000, 10, 100, 20, 1000, 500, 5000, 0, np.nan]
df_train =order_num(df_train, "CompanySize", order_lis)
df_train["CompanySize"].unique()

array([10000.,    10.,   100.,    20.,  1000.,   500.,  5000.,     0.,
          nan])

In [11]:
#カテゴリ特徴量の変換　https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/
import category_encoders as ce


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
#欠損値補完の前に欠損値カウント
def null_count(df):
  df["null_count"] = df.isnull().sum(axis=1)
  return df

df_train = null_count(df_train)
df_train.head()

,Respondent,ConvertedSalary,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,...,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy,null_count
0,1073.36,70841.0,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...",10000.0,Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,...,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,3
1,1073.36,21426.0,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",10000.0,Data or business analyst;Desktop or enterprise...,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,...,1.0,7.0,5.0,Algorithms making important decisions,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, but only within the company",Upper management at the company/organization,Yes,10 (Very Likely),Daily or almost daily,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Before 5:00 AM,Over 12 hours,1 - 2 hours,Never,NaN,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,6
2,1073.36,41671.0,Yes,No,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",10.0,Back-end developer;Data

In [13]:
#数値型特徴量の欠損値補完



In [14]:
#数値型特徴量の標準化　https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
from sklearn.preprocessing import StandardScaler



In [15]:
#テキストをElastic-Netで

In [ ]:
#パーティショニング (層化抽出)